In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

In [ ]:
def calculate_and_plot_average_for_participant(participant, button_sizes, output_dir, num_trials=3, threshold_distance=0.04):
    # Dictionary to store the average error rates for each button size
    error_rates = {}

    sns.set(style="darkgrid")

    # Define a list of colors for plotting different trials
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

    # Create a figure with a grid layout depending on the number of button sizes
    grid_rows, grid_cols = 2, int(np.ceil(len(button_sizes) / 2))
    fig = plt.figure(figsize=(grid_cols * 6, grid_rows * 6))
    fig.suptitle(f'Participant {participant}', fontsize=32)

    for idx, button_size in enumerate(button_sizes):
        aggregated_df = pd.DataFrame()
        trial_errors = []

        ax = fig.add_subplot(grid_rows, grid_cols, idx + 1, projection='3d')

        for trial in range(1, num_trials + 1):
            csv_path = os.path.join(output_dir, f'P{participant}_T{trial}_{button_size}.csv')
            df = pd.read_csv(csv_path)
            aggregated_df = aggregated_df.append(df)

            # Reference point - the last point in the dataset
            ref_point = df.iloc[-1]

            # Filter points based on the threshold distance (Finding points that signal pressing attempts)
            distances = np.sqrt((df['PositionX'] - ref_point['PositionX'])**2 +
                                (df['PositionY'] - ref_point['PositionY'])**2 +
                                (df['PositionZ'] - ref_point['PositionZ'])**2)
            filtered_points = df[distances < threshold_distance]

            # Store the average error distance for this trial
            trial_errors.append(distances[distances < threshold_distance].mean())

            # Plot this trial's filtered data
            color = colors[trial % len(colors)]  # Cycle through colors
            ax.scatter(filtered_points['PositionX'], filtered_points['PositionY'], filtered_points['PositionZ'], c=color, marker='o', label=f'Trial {trial}')

            # Plot the reference point with a distinct marker and size
            ax.scatter(ref_point['PositionX'], ref_point['PositionY'], ref_point['PositionZ'], c='black', marker='x', s=100, label=f'Ref Point Trial {trial}' if trial == 1 else '')

        # Calculate the average error rate across all trials for this button size
        average_error = np.mean(trial_errors)
        error_rates[button_size] = average_error

        # Set plot labels and title
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Button Size: {button_size}mm')
        ax.text2D(0.05, 0.95, f'Avg Error: {average_error:.2f}', transform=ax.transAxes)
        ax.legend()

    # Adjust layout for padding between subplots
    plt.tight_layout(pad=3.0)

    # Save the entire figure containing all subplots
    plt.savefig(os.path.join(output_dir, f'P{participant}_average_plot.png'))
    plt.close()

    # Save the error rates to a CSV file
    error_rates_df = pd.DataFrame(list(error_rates.items()), columns=['ButtonSize', 'AverageErrorRate'])
    error_rates_df.to_csv(os.path.join(output_dir, f'P{participant}_average_errors.csv'), index=False)

    return error_rates_df

# Example usage for a participant
button_sizes = ['15', '25', '35', '45', '55']
output_directory = '/content'
for subject in range(1, 11):
    error_rates_df = calculate_and_plot_average_for_participant(subject, button_sizes, output_directory)

In [4]:
def compile_error_rates(output_dir, num_participants, button_sizes):
    compiled_data = pd.DataFrame(index=range(1, num_participants + 1))

    for participant in range(1, num_participants + 1):
        file_path = os.path.join(output_dir, f'P{participant}_average_errors.csv')

        if os.path.exists(file_path):
            # Read the participant's error rates
            participant_data = pd.read_csv(file_path)

            # Set the participant's row in the compiled DataFrame
            for idx, row in participant_data.iterrows():
                button_size = row['ButtonSize']
                average_error = row['AverageErrorRate']
                compiled_data.at[participant, button_size] = average_error

    # Save the compiled data to a new CSV file
    compiled_file_path = os.path.join(output_dir, 'compiled_average_errors.csv')
    compiled_data.to_csv(compiled_file_path, index_label='Participant')

    return compiled_file_path

output_directory = '/content'
num_participants = 10
button_sizes = ['15', '25', '35', '45', '55']
compiled_file_path = compile_error_rates(output_directory, num_participants, button_sizes)

print(f"Compiled file created at: {compiled_file_path}")

Compiled file created at: /content/compiled_average_errors.csv
